In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import unicodedata

In [2]:
url = 'https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/cases-in-us.html'

### Extract date

In [3]:
content = requests.get(url).content
soup =BeautifulSoup(content, 'lxml')

In [4]:
def _removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

data = soup.findAll(text=re.compile(".*pdated .*202"))
data


['Last updated on May 1, 2020']

In [5]:
if len(data)>0:
    m = re.search(".*pdated .* (\w+ \d+, 202\d)", _removeNonAscii(data[0]))
    data_date_str = m.group(1)
    data_date = datetime.strptime(data_date_str, '%B %d, %Y').strftime('%Y-%m-%d')
else:
    data_date = datetime.utcnow().strftime('%Y-%m-%d')
data_date

'2020-05-01'

### Extract data and add into Dataframe

In [6]:
dfs = pd.read_html(url)

In [7]:
if len(dfs) == 0 :
    raise Exception('There is no dataframes detected')

In [8]:
dfs

[                           Unnamed: 0_level_0  \
                             Age group (years)   
 0                                      Totals   
 1                    Race missing/unspecified   
 2                              Race specified   
 3             Among those with race specified   
 4            American Indian or Alaska Native   
 5                                       Asian   
 6                   Black or African American   
 7   Native Hawaiian or other Pacific Islander   
 8                                       White   
 9                              Multiple/other   
 10                                        NaN   
 11              Ethnicity missing/unspecified   
 12                        Ethnicity specified   
 13       Among those with ethnicity specified   
 14                            Hispanic/Latino   
 15                        Non-Hispanic/Latino   
 
                No. of cases (% of total)  \
                                     < 18   
 0      

In [9]:
df = dfs[0]
df

Unnamed: 0_level_0  \
                            Age group (years)   
0                                      Totals   
1                    Race missing/unspecified   
2                              Race specified   
3             Among those with race specified   
4            American Indian or Alaska Native   
5                                       Asian   
6                   Black or African American   
7   Native Hawaiian or other Pacific Islander   
8                                       White   
9                              Multiple/other   
10                                        NaN   
11              Ethnicity missing/unspecified   
12                        Ethnicity specified   
13       Among those with ethnicity specified   
14                            Hispanic/Latino   
15                        Non-Hispanic/Latino   

               No. of cases (% of total)  \
                                    < 18   
0                                  17009   
1                          11,384(66.9%)   
2                           5,625(33.1%)   
3        Among those with race specified   
4                              100(1.8%)   
5                              253(4.5%)   
6                           1,341(23.8%)   
7                               25(0.4%)   
8                           2,831(50.3%)   
9                           1,075(19.1%)   
10                                   NaN   
11                         11,270(66.3%)   
12                          5,739(33.7%)   
13  Among those with ethnicity specified   
14                          2,288(39.9%)   
15                          3,451(60.1%)   

                                          \
                                   18-44   
0                                 283190   
1                         178,460(63.0%)   
2                         104,730(37.0%)   
3        Among those with race specified   
4                            1,013(1.0%)   
5                            5,582(5.3%)   
6                          28,821(27.5%)   
7                              425(0.4%)   
8                          52,012(49.7%)   
9                          16,877(16.1%)   
10                                   NaN   
11                        185,698(65.6%)   
12                         97,492(34.4%)   
13  Among those with ethnicity specified   
14                         30,778(31.6%)   
15                         66,714(68.4%)   

                                          \
                                   45-64   
0                                 279653   
1                         164,292(58.7%)   
2                         115,361(41.3%)   
3        Among those with race specified   
4                              870(0.8%)   
5                            6,142(5.3%)   
6                          35,977(31.2%)   
7                              325(0.3%)   
8                          56,383(48.9%)   
9                          15,664(13.6%)   
10                                   NaN   
11                        175,626(62.8%)   
12                        104,027(37.2%)   
13  Among those with ethnicity specified   
14                         26,423(25.4%)   
15                         77,604(74.6%)   

                                          \
                                   65-74   
0                                  87403   
1                          42,467(48.6%)   
2                          44,936(51.4%)   
3        Among those with race specified   
4                              234(0.5%)   
5                            2,092(4.7%)   
6                          14,434(32.1%)   
7                              100(0.2%)   
8                          23,631(52.6%)   
9                            4,445(9.9%)   
10                                   NaN   
11                         48,810(55.8%)   
12                         38,593(44.2%)   
13  Among those with ethnicity specified   
14                          6,902(17.9%)   
15               

In [10]:
csv_file='../data/cdc-us-cases-by-races-and-age-group.csv'
df.to_csv(csv_file, index=False)

In [11]:
# col0 = df.columns[0]
# df = df.set_index(col0).T
# df



In [12]:
# # remove non alphanumeric characters from the columns
# # also lowercase them
# df.columns = df.columns.map(lambda x: re.sub('\W+', '', str.lower(x)))

# # add column date to the front
# df.insert(0, 'date', data_date)


### Save to file (old data)


In [13]:
# from pathlib import Path
# csv_file = '../data/cdc-us-cases.csv'

# if Path(csv_file).exists():
#     # read out the old dat
#     old_df = pd.read_csv(csv_file)
#     df = df.append(old_df)
# df.drop_duplicates(subset=['date'], inplace=True)

In [14]:
# df.sort_values(by=['date'], inplace=True)
# df
# df.to_csv(csv_file, index=False)